# ⚡ Eletroniquiz - Versão Final Com Estilo
Aprendizado por níveis, com interface amigável, botões interativos e reforço educativo.

## 📌 Como usar:
1. Faça upload do arquivo `perguntas_chatbot_niveis_completas.json`
2. Execute a célula abaixo para iniciar o quiz
3. As perguntas aparecem por ordem de dificuldade (fácil → difícil)
4. Clique nos botões para responder — tudo interativo!

In [ ]:
import json
import random
from IPython.display import display, clear_output, Markdown
import ipywidgets as widgets

# Variáveis principais
score = 0
pulos = 2
acertos_consecutivos = 0
erros_totais = 0
perguntas_feitas = 0
total_questoes = 40
pendentes = []

# Carrega perguntas em ordem crescente de dificuldade
with open("perguntas_chatbot_niveis_completas.json", "r", encoding="utf-8") as f:
    perguntas_por_nivel = json.load(f)

todas_perguntas = []
for nivel in ["1", "2", "3"]:
    todas_perguntas.extend(perguntas_por_nivel[nivel])

# Score e status
def barra_status():
    blocos = score // 5
    return f"📊 SCORE: [{'█'*blocos}{'-'*(20 - blocos)}] {score}/100 | 🌀 PULOS: {pulos} | ✅ ACERTOS: {acertos_consecutivos} | ❌ ERROS: {erros_totais}"

# Mostra pergunta
def mostrar_pergunta(pergunta):
    global score, pulos, acertos_consecutivos, perguntas_feitas, erros_totais

    clear_output(wait=True)
    print(f"🔧 PERGUNTA {perguntas_feitas + 1}/{total_questoes}")
    print(barra_status())
    print("\n")  # Espaçamento extra

    # Exibir pergunta com fonte maior
    display(Markdown(f"<span style='font-size:20px;'>❓ {pergunta['pergunta']}</span>"))

    botoes = []
    for opcao in pergunta['opcoes']:
        label = opcao.lower()
        btn = widgets.Button(description=label,
                             layout=widgets.Layout(width='250px', align_self='flex-start'))
        def ao_clicar(b, escolha=label[0], correta=pergunta['resposta'].lower(), p=pergunta):
            global score, acertos_consecutivos, pulos, perguntas_feitas, erros_totais
            if escolha == correta:
                score += 5
                acertos_consecutivos += 1
                print("✅ Correto! +5 pontos.")
            else:
                score = max(0, score - 3)
                acertos_consecutivos = 0
                erros_totais += 1
                pendentes.append(p)
                print(f"❌ Errado! -3 pontos. Resposta correta era: {correta}")
            if acertos_consecutivos > 0 and acertos_consecutivos % 10 == 0:
                pulos += 1
                print("🎁 Bônus: +1 pulo por 10 acertos consecutivos!")
            perguntas_feitas += 1
            proxima()
        btn.on_click(ao_clicar)
        botoes.append(btn)

    botao_pular = widgets.Button(description="⏭️ PULAR", button_style='warning',
                                 layout=widgets.Layout(width='250px', align_self='flex-start'))
    def pular(b):
        global pulos, score, perguntas_feitas
        if pulos > 0:
            pulos -= 1
            score = max(0, score - 1)
            pendentes.append(pergunta)
            print("⏭️ Pergunta pulada. -1 ponto.")
            perguntas_feitas += 1
            proxima()
        else:
            print("🚫 Sem pulos restantes.")
    botao_pular.on_click(pular)

    display(widgets.VBox(botoes + [botao_pular]))

# Próxima pergunta
def proxima():
    if perguntas_feitas < total_questoes and score < 100:
        mostrar_pergunta(todas_perguntas[perguntas_feitas])
    else:
        fim()

# Fim
def fim():
    clear_output()
    print("🏁 FIM DA SESSÃO!")
    print(barra_status())
    if score >= 100:
        print("🌟 PARABÉNS! Você atingiu o nível mestre!")
    else:
        print("💡 Continue praticando e use o reforço para melhorar!")

# Início
proxima()
